In [15]:
import openai
import csv
import os
from dotenv import load_dotenv
import requests
import pandas as pd


In [16]:
def read_questions(file_path):
    with open(file_path, 'r') as file:
        questions = file.readlines()
    return [q.strip() for q in questions]

In [17]:
def get_gpt4_response(question):
  response = openai.ChatCompletion.create(
    model="gpt-4-0125-preview",  # cheapest and newest model right now
    messages=[{"role": "user", "content": question}]
  )
  print(response.choices[0].message['content'].strip())
  return response.choices[0].message['content'].strip()

In [18]:
def political_api_response(text, api_key):
    url = "https://api.thebipartisanpress.com/api/endpoints/beta/robert/"
    setup = {"API": api_key, "Text": text}
    response = requests.post(url, data=setup)
    print(response.text)
    return response.text


In [19]:
def write_response_to_csv(question, response, score, csv_file):
    # Open the file in append mode ('a') to add each row without overwriting
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([question, response, score])

In [20]:
def main():
    load_dotenv()
    openai.api_key = os.getenv('OPENAI_API_KEY')
    bias_api_key = os.getenv('BIAS_API_KEY')

    questions = read_questions('questions.txt')
    csv_file = 'gpt-4-0125-preview_responses.csv'
    
    # Initialize the CSV file with headers before processing questions
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Question', 'Response', 'Score'])
    
    for question in questions:
        response = get_gpt4_response(question)
        score = political_api_response(response, bias_api_key)
        write_response_to_csv(question, response, score, csv_file)

In [21]:
main()

The belief that abortion is inherently immoral is rooted in various religious, philosophical, ethical, and cultural perspectives. These beliefs are complex and multi-faceted, often influenced by how individuals and societies view human life, personhood, rights, and morality. Here are some common reasons why some people consider abortion inherently immoral:

1. **Sanctity of Life:** Many argue from a standpoint that all human life is sacred or has intrinsic value, from conception to natural death. This belief is often grounded in religious teachings. For example, many Christians, Muslims, and followers of other religions believe that life is a gift from God and only God has the authority to end it. Therefore, ending a pregnancy through abortion is viewed as taking an innocent life, equating it with murder.

2. **Personhood at Conception:** Some people believe that a human embryo or fetus is a person with rights from the moment of conception. From this perspective, abortion is seen as a 

KeyboardInterrupt: 